<a href="https://colab.research.google.com/github/spider-tronix/Dark-Sight/blob/colab_config/ds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#0M NAMO NARAYANA

In [2]:
!ls

drive  sample_data


In [3]:
import os
os.listdir()
try:
  os.listdir().find('ds')
  first_time = False
except:
  first_time = True

In [4]:
if (first_time):
  !mkdir ds
  os.chdir('ds')
  os.listdir()

In [5]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
target_datasetdir = '/content/drive/My Drive/Dataset/Dataset/'
target_path = {
    'dataset': '/content/drive/My Drive/Dataset/Dataset/', 
    'save': '/content/drive/My Drive/Dataset/Dataset/results/',
    'checkpoint': '/content/drive/My Drive/Dataset/Colab/checkpoints/'
    }

In [ ]:
if (first_time):
  !git clone https://github.com/spider-tronix/Dark-Sight.git
  os.chdir('./Dark-Sight')
else:
  !git pull
print(os.listdir())
!git checkout colab_config
!git status
!git log
first_time = False
print(os.listdir())

In [8]:
!python ./data/darksight_dataloader.py

2021-02-06 02:34:17.611926: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
no. of datapoints 0
Traceback (most recent call last):
  File "./data/darksight_dataloader.py", line 241, in <module>
    data = DarkSighDataLoader().load()
  File "./data/darksight_dataloader.py", line 236, in load
    dataloader = DataLoader(self.transformed_dataset, batch_size, shuffle=shuffle)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 262, in __init__
    sampler = RandomSampler(dataset, generator=generator)  # type: ignore
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/sampler.py", line 104, in __init__
    "value, but got num_samples={}".format(self.num_samples))
ValueError: num_samples should be a positive integer value, but got num_samples=0


In [ ]:
# OM NAMO NARAYANA


"""run on base dir"""


import numpy as np
import torch.optim as optim
import torch
import torch.nn as nn
import sys
from torch.utils.data import DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os
import errno

sys.path.insert(1, "./")
from DarkNet.models.sid_unet import sidUnet
from DarkNet.models.lavi_unet import laviUnet
from data.darksight_dataloader import DarkSighDataLoader


"""Parameters"""

batch_size = 2

# set to True if thermal response included
inc_therm = False
load_weights = True

# change the directory for required model
checkpoint_dir = target_path['checkpoint']
save_dir = target_path['save']
raw_format = False
model = laviUnet(inc_therm=inc_therm, raw_format=raw_format)
plot_freq = 10
error_freq = 5
checkpoint_freq = 25
dataset_dir = target_path['dataset']

epochs = 500
lr = 1e-4


"""training code"""
os.chdir("./")
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.L1Loss()


"""creating necessary directories"""

if not raw_format:
    save_dir = save_dir + "jpg_format/"
    checkpoint_dir = checkpoint_dir + "jpg_format/"

if not inc_therm:
    save_dir = save_dir + "without_therm/"
    checkpoint_dir = checkpoint_dir + "without_therm/"

try:
    os.makedirs(save_dir)
    print(save_dir + " created")
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

try:
    os.makedirs(checkpoint_dir)
    print(checkpoint_dir + " created")
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

if not load_weights or os.listdir(checkpoint_dir) == []:
    if load_weights:
        print("no models saved")
    epoch_loaded = 0
else:
    checkpoint = torch.load(checkpoint_dir + os.listdir(checkpoint_dir)[-1])
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    epoch_loaded = checkpoint["epoch"]
    print("model loaded")

trainloader = DarkSighDataLoader(
    inc_therm=inc_therm, raw_format=raw_format, dataset_dir=dataset_dir
).load(batch_size=batch_size)

for epoch in range(epoch_loaded, epoch_loaded + epochs):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, gt = data
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = torch.transpose(outputs, 1, 3)
        outputs = torch.transpose(outputs, 1, 2)
        loss = criterion(outputs, gt)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        print(
                "epoch: %d, iteration: %d runningloss: %.3f" % (epoch + 1, i + 1, running_loss / 2)
            )

        """saving results"""
        if i % batch_size == 1 and epoch % error_freq == 1:
            print(
                "epoch: %d, iteration: %d loss: %.3f" % (epoch + 1, i + 1, running_loss / 2)
            )
            running_loss = 0.0
            f, (ax1, ax2, ax3) = plt.subplots(1, 3)
            ax1.imshow(outputs.detach().numpy()[0])
            ax2.imshow(gt.detach().numpy()[0])
            ax3.imshow(
                torch.transpose(torch.transpose(inputs.detach(), 1, 3), 1, 2).numpy()[
                    0
                ][:, :, :3]
            )

            plt.savefig(save_dir + "epoch{}_index{}.png".format(epoch + 1, i + 1))

        """saving checkpoints"""
        if i % batch_size == 1 and epoch % checkpoint_freq == 1:

            torch.save(
                {
                    "epoch": epoch,
                    "model_state_dict": model.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "loss": loss,
                },
                checkpoint_dir + str(int(epoch / checkpoint_freq) + 1) + ".tar",
            )

            print(
                "checkpoint %d_%d.tar saved."
                % (int(epoch / checkpoint_freq) + 1, i + 1)
            )


print("Finished Training")
# plt.show()


no models saved
no. of datapoints 147
data index 126
sampled on 11 iteration
data index 99
sampled on 4 iteration
epoch: 1, iteration: 1 runningloss: 0.211
data index 42
sampled on 5 iteration
data index 134
sampled on 2 iteration
epoch: 1, iteration: 2 runningloss: 0.432
data index 84
sampled on 2 iteration
data index 32
sampled on 11 iteration
epoch: 1, iteration: 3 runningloss: 0.657
data index 60
sampled on 1 iteration
data index 133
sampled on 6 iteration
epoch: 1, iteration: 4 runningloss: 0.877
data index 76
sampled on 1 iteration
data index 66
sampled on 3 iteration
epoch: 1, iteration: 5 runningloss: 1.093
data index 132
sampled on 8 iteration
data index 6
sampled on 1 iteration
epoch: 1, iteration: 6 runningloss: 1.312
data index 143
sampled on 3 iteration
data index 70
sampled on 11 iteration
epoch: 1, iteration: 7 runningloss: 1.531
data index 11
sampled on 1 iteration
data index 13
sampled on 1 iteration
epoch: 1, iteration: 8 runningloss: 1.755
data index 123
sampled on 1